# Lab 6 - Operational Fairness in DA/ML
Week 6 - Q3, 22/23 <br>
SEN163B: Responsible Data Analytics<br>
 

By <b> Nadia Metoui* </b> <br>
TA <b> Darsh Modi</b> <br> 
Faculty of Technology, Policy, and Management (TPM)<br>

***Learning Objectives***<br>
At the end of this lab, you will be able to 

- Use data analytics tools to measure disaparities in a Data Analytics Project
- Use data analytics tools to mitigate disaparities in a Data Analytics Project


***Structure***
- Part I. Measuring Disparities with Aequitas (See notebook for Part I)
- Part II Mitigating Bias/Disparities with FAI360 
- Part III: Mitigating Bias/Disparities with FairLearn (a tutorial will be provided for homework exploration)


##Part II: Mitigating Bias with FAI360

*Acknowledgement: Part II of this assignment is loosely based on the code developed by <i><b>Agathe Balayn</b></i> and <i><b>Seda Gürses</b></i>

In this part of the lab you will be requested to take a closer look at the data and identify and mitigate biases. We will use some tools form <i><b>AIF360 toolkit</b></i><br>(https://aif360.mybluemix.net/) a toolkit developed by IBM to detect and mitigate "bias" and "unfairness".

We will mainely focus on the pakages:
- `aif360.metrics.BinaryLabelDatasetMetric`: Metrics used to assess bias in the dataset before training. You will see examples with `mean_difference` and `disparate_impact`
- `aif360.metrics.ClassificationMetric`: Metrics used to assess bias in outcomes of the model You will see examples with `mean_difference` and `disparate_impact` and `false_discovery_rate_ratio`
- `aif360.algorithms.preprocessing` Bias mitigating algorithm to be applied to a data set before training the model. You will see an example with the `Reweighing` algorithm. 

You are encouraged to explore AIF360 Library more and use adequate metrics and algorithmes in your analysis byond what we see in this Lab. yu can access AIF360 library documentation [HERE](https://aif360.readthedocs.io/en/stable/index.html) and the AIF360 project page [HERE](https://aif360.mybluemix.net/). Take a close look to both!

Steps of Part II
- Step 0: Set-up (Provided)
- Step 1: Pre-processing Biases: Identify and Mitigate
- Step 2: In-processing Biases: Identify and Mitigate

### Setp 0: Set-up

You first need to install the required libraries for this part.  The main libraries are the `aif360` and `sklearn` ones. We also recommend using `numpy` or `pandas` to easily manipulate and explore the data.

<div class="alert alert-block alert-danger">
<b>Note:</b> Uncomment and run the next cell if you have not previously installed the libraries.
</div>


<b>Installing required libraries</b>

In [1]:
# !pip install aif360
# !pip install tensorflow

<b>Loading required libraries</b>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Markdown, display


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

np.random.seed(0)

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import MetaFairClassifier
from aif360.explainers import MetricTextExplainer, MetricJSONExplainer

from aif360.datasets.lime_encoder import LimeEncoder

import json
from collections import OrderedDict



pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


**Option 1 Google Colab:**<br>
Uncomment the following cell to download the dataset in google colab.

In [3]:
#Download the German Credit DataSet
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
!cp german.data /usr/local/lib/python3.9/dist-packages/aif360/data/raw/german/german.data
!cp german.doc /usr/local/lib/python3.9/dist-packages/aif360/data/raw/german/german.doc

--2023-03-22 18:47:29--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79793 (78K) [application/x-httpd-php]
Saving to: ‘german.data.22’

german.data.22      100%[===================>]  77.92K  --.-KB/s    in 0.1s    

2023-03-22 18:47:30 (604 KB/s) - ‘german.data.22’ saved [79793/79793]

--2023-03-22 18:47:30--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4679 (4.6K) [application/x-httpd-php]
Saving to: ‘german.doc.22’

german.doc.22       100%[===================>]   4.57K  --.-KB/s  

**Option 2: Local environment**<br>
<div class="alert alert-block alert-danger">
<b>Note:</b> If you are working on your local environment you will have to manually add the files "german.doc" and "german.data" to the folder 
"dist-packages/aif360/data/raw/german/" under your python path.<br> 
(or write your script to do it deppending on the os/platform you are using)
You can find the files in the lab folder on github or download them from: <br>
<a href="https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data">https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data</a> <br>
<a href="https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc">https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc</a>
</div> 

In [4]:
# Write your script here

### Step 1: Pre-Prosessing Biases 

The dataset is already encoded as the algorithms need the dataset to have numerical values and not categorical.

In [5]:
dataset_orig = GermanDataset(protected_attribute_names=['age'],           # this dataset also contains protected
                                                                          # attribute for "sex" which we do not
                                                                          # which we do not consider in this evaluation 
                                                                          # as well as its proxy personal_status
                             privileged_classes=[lambda x: x >= 25],      #age >=25 is considered privileged
                             features_to_drop=['personal_status', 'sex']) # ignore sex-related attributes and proxies

In [6]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

#### Compute fairness metric on original training dataset
In Lab 4, Week 4 you identified 'age' as one of the protected attributes in the german dataset and you defined a privileged and unprivileged categories, we can now use aif360 to detect bias in the dataset.  



##### Mean Difference 

**Definition:** Mean Difference Compares the percentage of favorable results (in the dataset) for the privileged and unprivileged groups, subtracting the former percentage from the latter.

The ideal value of this metric is 0.

A value < 0 indicates less favorable outcomes for the unprivileged groups.  This is implemented in the method called mean_difference on the BinaryLabelDatasetMetric class.  The code below performs this check and displays the output, showing that the difference is -0.169905.

In [7]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905


##### Disparate Impact (in Data Labeling)
**Definition:** Disparate Impact  is computed as the ratio of rate of favorable outcome (in the dataset) for the unprivileged group to that of the privileged group. The ideal value of this metric is 1.0.

A value < 1 implies higher benefit for the privileged group and a value >1 implies a higher benefit for the unprivileged group.

In [8]:
display(Markdown("#### Original training dataset"))
print("Disparate Impact = %f" % metric_orig_train.disparate_impact())

#### Original training dataset

Disparate Impact = 0.766430


##### Explainers

These briefly explain what a metric is and/or how it is calculated unless it is obvious (e.g. accuracy) and print the value.

This class contains text `MetricTextExplainer` or JSON `MetricJSONExplainer` formatted explanations for all metric values regardless of which subclass they appear in. This will raise an error if the metric does not apply (e.g. calling true_positive_rate if type(metric) == DatasetMetric).

***Text Explanations***

In [9]:
text_expl = MetricTextExplainer(metric_orig_train)
json_expl = MetricJSONExplainer(metric_orig_train)

In [10]:
print(text_expl.mean_difference())

Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.1699054740619017


In [11]:
print(text_expl.disparate_impact())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.7664297113013201


***JSON Explanations***

In [12]:
def format_json(json_str):
    return json.dumps(json.loads(json_str, object_pairs_hook=OrderedDict), indent=2)

In [13]:
print(format_json(json_expl.mean_difference()))

{
  "metric": "Mean Difference",
  "message": "Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.1699054740619017",
  "numPositivesUnprivileged": 63.0,
  "numInstancesUnprivileged": 113.0,
  "numPositivesPrivileged": 427.0,
  "numInstancesPrivileged": 587.0,
  "description": "Computed as the difference of the rate of favorable outcomes received by the unprivileged group to the privileged group.",
  "ideal": "The ideal value of this metric is 0.0"
}


In [14]:
print(format_json(json_expl.disparate_impact()))

{
  "metric": "Disparate Impact",
  "message": "Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.7664297113013201",
  "numPositivePredictionsUnprivileged": 63.0,
  "numUnprivileged": 113.0,
  "numPositivePredictionsPrivileged": 427.0,
  "numPrivileged": 587.0,
  "description": "Computed as the ratio of rate of favorable outcome for the unprivileged group to that of the privileged group.",
  "ideal": "The ideal value of this metric is 1.0 A value < 1 implies higher benefit for the privileged group and a value >1 implies a higher benefit for the unprivileged group."
}


#### Mitigate bias by transforming the original dataset
The previous step showed that the privileged group was getting 17% more positive outcomes in the training dataset.   Since this is not desirable, we are going to try to mitigate this bias in the training dataset.  As stated above, this is called _pre-processing_ mitigation because it happens before the creation of the model.  

AI Fairness 360 implements several pre-processing mitigation algorithms.  We will choose the Reweighing algorithm [1], which is implemented in the `Reweighing` class in the `aif360.algorithms.preprocessing` package.  This algorithm will transform the dataset to have more equity in positive outcomes on the protected attribute for the privileged and unprivileged groups.

We then call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (dataset_transf_train).

`[1] F. Kamiran and T. Calders,  "Data Preprocessing Techniques for Classification without Discrimination," Knowledge and Information Systems, 2012.`

##### ***Reweighing***

Reweighing is a data preprocessing technique that recommends generating weights for the training examples in each (group, label) combination differently to ensure fairness before classification. The idea is to apply appropriate weights to different tuples in the training dataset to make the training dataset discrimination free with respect to the sensitive attributes. Instead of reweighing, one could also apply techniques (non-discrimination constraints) such as suppression (remove sensitive attributes) or massaging the dataset — modify the labels (change the labels appropriately to remove discrimination from the training data). However, the reweighing technique is more effective than the other two mentioned earlier.

In [15]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

In [16]:
dataset_transf_train.instance_weights

array([0.96229508, 0.96229508, 0.96229508, 0.96229508, 0.96229508,
       0.96229508, 0.96229508, 0.96229508, 1.25555556, 0.678     ,
       1.100625  , 1.100625  , 0.96229508, 0.96229508, 1.100625  ,
       0.96229508, 1.25555556, 1.100625  , 0.96229508, 0.96229508,
       0.96229508, 0.96229508, 0.96229508, 0.96229508, 0.96229508,
       1.100625  , 0.96229508, 0.96229508, 0.96229508, 0.678     ,
       0.96229508, 0.96229508, 0.678     , 1.100625  , 0.96229508,
       0.678     , 0.96229508, 0.96229508, 1.100625  , 0.96229508,
       1.100625  , 0.96229508, 0.96229508, 1.25555556, 0.96229508,
       0.678     , 1.100625  , 0.96229508, 0.96229508, 1.25555556,
       1.100625  , 1.100625  , 0.96229508, 0.96229508, 1.100625  ,
       0.96229508, 0.96229508, 0.96229508, 0.96229508, 0.96229508,
       1.100625  , 0.96229508, 0.96229508, 0.96229508, 0.96229508,
       0.96229508, 0.96229508, 1.100625  , 0.678     , 0.96229508,
       0.96229508, 0.96229508, 0.96229508, 0.96229508, 1.25555

In [17]:
len(dataset_transf_train.instance_weights)

700

#### Compute fairness metric on transformed dataset
Now that we have a transformed dataset, we can check how effective it was in removing bias by using the same metric we used for the original training dataset in Step 3.  Once again, we use the function mean_difference in the BinaryLabelDatasetMetric class.   We see the mitigation step was very effective, the difference in mean outcomes is now 0.0.  So we went from a 17% advantage for the privileged group to equality in terms of mean outcome.

##### Mean Difference 

In [18]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


##### Disparate Impact

In [19]:
display(Markdown("#### Transformed training dataset"))
print("Disparate Impact = %f" % metric_transf_train.disparate_impact())

#### Transformed training dataset

Disparate Impact = 1.000000


### Step 3: In-Prosessing Biases 

<b>Preparation for training a classifier.</b><br>
We will  divide the dataset into a training and a test subsets.
We define them respectively as 70% and 30% of the whole data.
We will use the following code to do so.

In [20]:
dataset_gcredit_train, dataset_gcredit_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

#### Compute Faireness of the model before mitigation
Below we are using several faireness metrics from AIF360 toolkint to evaluate fairenesse metrics before appling the inprocessing mitigation

Build a classifier without fairness constraints (tau penalty = 0 see [MetaFairClassifier documentation](https://aif360.readthedocs.io/en/stable/modules/generated/aif360.algorithms.inprocessing.MetaFairClassifier.html#aif360.algorithms.inprocessing.MetaFairClassifier))

In [21]:
biased_model = MetaFairClassifier(tau=0, sensitive_attr="age", type="fdr").fit(dataset_gcredit_train)

Apply the unconstrained model to test data

In [22]:
dataset_bias_test = biased_model.predict(dataset_gcredit_test)

Test the "biased" classifier

In [23]:
metric_orig_test = BinaryLabelDatasetMetric(dataset_gcredit_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = {:.3f}".format(metric_orig_test.mean_difference()))

Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.144


In [24]:
classified_metric_bias_test = ClassificationMetric(dataset_gcredit_test, dataset_bias_test,
                                                   unprivileged_groups=unprivileged_groups,
                                                   privileged_groups=privileged_groups)
print("Test set: Classification accuracy = {:.3f}".format(classified_metric_bias_test.accuracy()))
TPR = classified_metric_bias_test.true_positive_rate()
TNR = classified_metric_bias_test.true_negative_rate()
bal_acc_bias_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = {:.3f}".format(bal_acc_bias_test))
print("Test set: Disparate impact = {:.3f}".format(classified_metric_bias_test.disparate_impact()))
fdr = classified_metric_bias_test.false_discovery_rate_ratio()
fdr = min(fdr, 1/fdr)
print("Test set: False discovery rate ratio = {:.3f}".format(fdr))

Test set: Classification accuracy = 0.537
Test set: Balanced classification accuracy = 0.440
Test set: Disparate impact = 0.952
Test set: False discovery rate ratio = 0.634


<div class="alert alert-block alert-info">
<b> Tip: </b> Make use of the documentation of <a href=https://aif360.readthedocs.io/en/latest/index.html><b>AIF360</b></a>, and your own search to understaind the metrics and to be able to interpret them.
</div>

In the following we will apply an in-processing bias mitigation technique <b><i>"Meta-Algorithm for fair classification"</i></b>. This technique operates with a faireness constraint i.e., by optimising for faireness metrics. You can read more about it <a href=https://arxiv.org/pdf/1806.06055.pdf>[HERE]</a><br>
For this example we will to optimize for the ***the fals discovery rate (fdr)*** and sensitive attribute ***age***

(Optional) You can also try **statistical rate/disparate impact (sr)** and sensitive attribute ***age***


#### Optimising for False Discovery Rate (FDR) and Re-compute Faireness

    
A. Apply the debiased model to training data and train the classifier with the selected fairness constrain

In [25]:
fdr_optimised_model = MetaFairClassifier(tau=0.5, sensitive_attr="age", type="fdr").fit(dataset_gcredit_train)

B. Apply the debiased classifier to test data

In [26]:
fdr_optimised_prediction = fdr_optimised_model.predict(dataset_gcredit_test)

C. Compute the same faireness metrics for new classifier (with fairness constraint)

In [27]:
fdr_optimised_metric = BinaryLabelDatasetMetric(fdr_optimised_prediction, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = {:.3f}".format(fdr_optimised_metric.mean_difference()))

Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.010


In [28]:
fdr_optimised_classification = ClassificationMetric(dataset_gcredit_test, 
                                                 fdr_optimised_prediction,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = {:.3f}".format(fdr_optimised_classification.accuracy()))
TPR = fdr_optimised_classification.true_positive_rate()
TNR = fdr_optimised_classification.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = {:.3f}".format(bal_acc_debiasing_test))
print("Test set: Disparate impact = {:.3f}".format(fdr_optimised_classification.disparate_impact()))
fdr = fdr_optimised_classification.false_discovery_rate_ratio()
fdr = min(fdr, 1/fdr)
print("Test set: False discovery rate ratio = {:.3f}".format(fdr))


Test set: Classification accuracy = 0.520
Test set: Balanced classification accuracy = 0.426
Test set: Disparate impact = 0.986
Test set: False discovery rate ratio = 0.566


#### Optimising for Disaparate Impact (SR) and Re-compute Faireness

    
A. Apply the debiased model to training data and train the classifier with the selected fairness constrain. The value of `tau` should be different from `0` We choose thos value after several tests

In [29]:
sr_optimised_model = MetaFairClassifier(tau=0.3, sensitive_attr="age", type="sr").fit(dataset_gcredit_train)

B. Apply the debiased classifier to test data

In [30]:
sr_optimised_predictions = sr_optimised_model.predict(dataset_gcredit_test)

C. Compute the same faireness metrics for new classifier (with fairness constraint)

In [31]:
sr_optimised_metric = BinaryLabelDatasetMetric(sr_optimised_predictions, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = {:.3f}".format(sr_optimised_metric.mean_difference()))

Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.121


In [32]:
classified_metric_debiasing_test = ClassificationMetric(dataset_gcredit_test, 
                                                 sr_optimised_predictions,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = {:.3f}".format(classified_metric_debiasing_test.accuracy()))
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = {:.3f}".format(bal_acc_debiasing_test))
print("Test set: Disparate impact (Classification) = {:.3f}".format(classified_metric_debiasing_test.disparate_impact()))
fdr = classified_metric_debiasing_test.false_discovery_rate_ratio()
fdr = min(fdr, 1/fdr)
print("Test set: False discovery rate ratio = {:.3f}".format(fdr))


Test set: Classification accuracy = 0.450
Test set: Balanced classification accuracy = 0.369
Test set: Disparate impact (Classification) = 1.203
Test set: False discovery rate ratio = 0.484


<br><br>